<a href="https://colab.research.google.com/github/kolayn808/textattack_test/blob/main/TextBugger_attack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Импорт необходимых библиотек

In [2]:
import numpy as np
import random
import nltk
from nltk.corpus import stopwords

Загрузка словаря стоп-слов

In [3]:
nltk.download('stopwords')

# Загрузка списка стоп-слов
stop_words = set(stopwords.words('english'))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Создание класса атаки TextBugger

In [7]:
class TextBugger_attacker:

    # Функция генерации багов в слове
    def generateBugs(word, bug_types={"insert", "delete", "swap", "sub_C"}):
        """
        Функция для генерации багов. Статья TextBugger
        Li J. et al. Textbugger: Generating adversarial text against real-world
        applications //arXiv preprint arXiv:1812.05271. – 2018.
        """

        bugs = {"insert": word, "delete": word, "swap": word, "sub_C": word, "sub_W": word}

        if (len(word) <= 2):
            return bugs

        # Генерация различных типов багов
        if "insert" in bug_types:
            bugs["insert"] = TextBugger_attacker.bug_insert(word)
        if "delete" in bug_types:
            bugs["delete"] = TextBugger_attacker.bug_delete(word)
        if "swap" in bug_types:
            bugs["swap"] = TextBugger_attacker.bug_swap(word)
        if "sub_C" in bug_types:
            bugs["sub_C"] = TextBugger_attacker.bug_sub_C(word)

        return bugs

    # Функция для вставки пробела в слово
    def bug_insert(word):
        if (len(word) >= 6):
            return word
        res = word
        point = random.randint(1, len(word) - 1)
        res = res[0:point] + " " + res[point:]
        return res

    # Функция для удаления буквы из слова
    def bug_delete(word):
        res = word
        point = random.randint(1, len(word) - 2)
        res = res[0:point] + res[point + 1:]
        return res

    # Функция для перемены местами двух соседних букв в слове
    def bug_swap(word):
        if (len(word) <= 4):
            return word
        res = word
        points = random.sample(range(1, len(word) - 1), 2)
        a = points[0]
        b = points[1]
        res = list(res)
        w = res[a]
        res[a] = res[b]
        res[b] = w
        res = ''.join(res)
        return res

    # Функция для замены буквы на визуально похожую
    def bug_sub_C(word):
        res = word
        key_neighbors = TextBugger_attacker.get_key_neighbors()
        point = random.randint(0, len(word) - 1)

        if word[point] not in key_neighbors:
            return word
        choices = key_neighbors[word[point]]
        subbed_choice = choices[random.randint(0, len(choices) - 1)]
        res = list(res)
        res[point] = subbed_choice
        res = ''.join(res)

        return res

    # Словарь соседних и похожих символов для использования при замене букв
    def get_key_neighbors():
        neighbors = {
            "q": "was", "w": "qeasd", "e": "wrsdf", "r": "etdfg", "t": "ryfgh", "y": "tughj", "u": "yihjk", "i": "uojkl",
            "o": "ipkl", "p": "ol",
            "a": "qwszx", "s": "qweadzx", "d": "wersfxc", "f": "ertdgcv", "g": "rtyfhvb", "h": "tyugjbn", "j": "yuihknm",
            "k": "uiojlm", "l": "opk",
            "z": "asx", "x": "sdzc", "c": "dfxv", "v": "fgcb", "b": "ghvn", "n": "hjbm", "m": "jkn"
        }

        neighbors['i'] += '1'
        neighbors['l'] += '1'
        neighbors['z'] += '2'
        neighbors['e'] += '3'
        neighbors['a'] += '@'
        neighbors['s'] += '5'
        neighbors['g'] += '6'
        neighbors['b'] += '8'
        neighbors['g'] += '9'
        neighbors['q'] += '9'
        neighbors['o'] += '0'

        return neighbors

    # Функция для генерации атаки на текст
    def textbugger_attack(original_text,
                          bug_nums=3,
                          bug_types={"insert", "delete", "swap", "sub_C"}):
        words = original_text.split(' ')
        sen_len = len(words)

        wn = set()
        adv_words = words

        for i in range(bug_nums):
            if len(wn) == sen_len:
                break

            # Выбор случайного слова в предложении для внесения изменений
            n = np.random.randint(low=0, high=sen_len)
            while n in wn or adv_words[n] in stop_words or len(adv_words[n]) <= 2:
                n = np.random.randint(low=0, high=sen_len)

            wn.add(n)
            candidate_word = adv_words[n]
            bugs_vars = bug_types.copy()

            # Исключение типов багов, которые неприменимы к текущему слову
            if len(candidate_word) <= 4:
                bugs_vars.difference_update({'swap', 'typoW'})
            elif len(candidate_word) >= 6:
                bugs_vars.remove('insert')

            # Генерация случайного типа бага
            #print(bugs_vars)
            bugs = TextBugger_attacker.generateBugs(candidate_word, bug_types=bugs_vars)
            at = random.choice(list(bugs_vars))
            print(f"{at}: {bugs[at]}")

            # Применение выбранного бага к слову в предложении
            adv_words = adv_words[:n] + [bugs[at]] + adv_words[n + 1:]
            adversarial_sentence = ' '.join(adv_words)
            print(original_text)
            print(adversarial_sentence)

            return adversarial_sentence

Пример атаки

In [8]:
original_text = "i really hoped for the best with this one , but it just did n't happen . financed at a very non - dutch manner and still looking great , with a style and pace that 's very much like hollywood"
adv_res = TextBugger_attacker.textbugger_attack(original_text, bug_nums=1, bug_types={"insert", "delete", "swap", "sub_C"})

sub_C: oike
i really hoped for the best with this one , but it just did n't happen . financed at a very non - dutch manner and still looking great , with a style and pace that 's very much like hollywood
i really hoped for the best with this one , but it just did n't happen . financed at a very non - dutch manner and still looking great , with a style and pace that 's very much oike hollywood
